# Import Libraries

In [8]:
install.packages("tidyverse", repos='http://cran.us.r-project.org')

also installing the dependencies ‘isoband’, ‘vroom’, ‘tzdb’, ‘broom’, ‘ggplot2’, ‘haven’, ‘lubridate’, ‘readr’, ‘tidyr’

Warning message in install.packages("tidyverse", repos = "http://cran.us.r-project.org"):
“installation of package ‘isoband’ had non-zero exit status”Warning message in install.packages("tidyverse", repos = "http://cran.us.r-project.org"):
“installation of package ‘tzdb’ had non-zero exit status”Warning message in install.packages("tidyverse", repos = "http://cran.us.r-project.org"):
“installation of package ‘lubridate’ had non-zero exit status”Warning message in install.packages("tidyverse", repos = "http://cran.us.r-project.org"):
“installation of package ‘tidyr’ had non-zero exit status”Warning message in install.packages("tidyverse", repos = "http://cran.us.r-project.org"):
“installation of package ‘vroom’ had non-zero exit status”Warning message in install.packages("tidyverse", repos = "http://cran.us.r-project.org"):
“installation of package ‘broom’ had non-zer

In [6]:
remove.packages("tidyverse")

Removing package from ‘/Users/devlish/opt/anaconda3/envs/Tartarus/lib/R/library’
(as ‘lib’ is unspecified)


ERROR: Error in find.package(pkgs, lib): there is no package called ‘tidyverse’


In [9]:
library("tidyverse")

ERROR: Error in library("tidyverse"): there is no package called ‘tidyverse’


# Import Files

In [1]:
test = read_csv("data/test.csv")
train = read_csv("data/train.csv")
sample = read_csv("data/sample_submission.csv")

ERROR: Error in read_csv("data/test.csv"): could not find function "read_csv"
